In [68]:
#%pip install -U spaCy
#%pip install keras
#%python -m spacy download en_core_web_sm
#%pip install -U jax jaxlib
#%pip install clean-text
#%pip install --upgrade ipykernel
#%pip install tensorflow
#%pip install numpy --upgrade
#%pip install numpy==1.21.6
#%pip uninstall tensorflow
#da eseguire al primo utilizzo

^C
Note: you may need to restart the kernel to use updated packages.


In [60]:
import string
import spacy
import json
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from spacy.lang.it.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from pandas import read_csv
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import svm
from sklearn import neural_network
from sklearn import ensemble
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from pandas import read_csv
import nltk
import random

#init spaCy
punctuations = string.punctuation
nlp = spacy.load("en_core_web_lg")
stop_words = spacy.lang.en.stop_words.STOP_WORDS
parser = English()

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    translator = str.maketrans("", "", string.punctuation)
    text_without_punctuation = text.translate(translator)
    return text_without_punctuation.lower()
    #return text.strip().lower()

# Tokenizer function
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.text for word in mytokens ]
    # remove stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    # return preprocessed list of tokens
    return mytokens


Number of CPUs in the system: 8


In [ ]:
import os
  
n_cpu = os.cpu_count()
print("Number of CPUs in the system:", n_cpu)

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
dft = read_csv('../Dataset/Processed datasets/train_sexist.csv')
dfv = read_csv('../Dataset/Processed datasets/dev_sexist.csv')
dfs = read_csv('../Dataset/Processed datasets/test_sexist.csv')

x_train = dft['text']
y_train = dft['label_sexist']
x_val = dfv['text']
y_val = dfv['label_sexist']
x_test = dfs['text']
y_test= dfs['label_sexist']
dft.set_index('ID')
dfv.set_index('ID')
dfs.set_index('ID')
print("TRAIN: \n", y_train.value_counts(), end="\n\n")
print("VALIDATION: \n", y_val.value_counts(), end="\n\n")
print("TEST: \n", y_test.value_counts(), end="\n\n")

TRAIN: 
 not sexist    10602
sexist         3398
Name: label_sexist, dtype: int64

VALIDATION: 
 not sexist    1514
sexist         486
Name: label_sexist, dtype: int64

TEST: 
 not sexist    3030
sexist         970
Name: label_sexist, dtype: int64



In [45]:
from transformers import BertTokenizer
from nltk.tokenize import MWETokenizer
from nltk.tokenize import TreebankWordTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
tokenizer1 = MWETokenizer()
tokenizer2 = TreebankWordTokenizer()
vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
vector6 = CountVectorizer(tokenizer = tokenizer2.tokenize, ngram_range=(1,2))

# **Task A**

In [46]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.utils import class_weight
import sklearn.metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, precision_score
from sklearn.neighbors import KNeighborsClassifier


classifier = svm.LinearSVC(max_iter = 10000, class_weight= {"not sexist": 0.2, "sexist": 1})
# Multi-layer Perceptron classifier
#classifier = neural_network.MLPClassifier(hidden_layer_sizes=(512,),verbose=True,max_iter=10)
# Random Foreset
#classifier = ensemble.RandomForestClassifier(class_weight='balanced')

# Create the pipeline

pipe = Pipeline([("cleaner", predictors()),
('vectorizer', vector6),
('classifier', classifier)])

pipe.fit(x_train, y_train)


Pipeline(steps=[('cleaner', <__main__.predictors object at 0x0000028D25F7B820>),
                ('vectorizer',
                 CountVectorizer(ngram_range=(1, 2),
                                 tokenizer=<bound method TreebankWordTokenizer.tokenize of <nltk.tokenize.treebank.TreebankWordTokenizer object at 0x0000028D267923A0>>)),
                ('classifier',
                 LinearSVC(class_weight={'not sexist': 0.2, 'sexist': 1},
                           max_iter=10000))])

In [47]:
y_pred = pipe.predict(x_val)

# Calcolo delle metriche di valutazione sul set di test
val_accuracy = accuracy_score(y_val, y_pred)
val_precision = precision_score(y_val, y_pred, average="macro")
val_recall = recall_score(y_val, y_pred, average="macro")
val_f1 = f1_score(y_val, y_pred, average="macro")

# Print the evaluation metrics for the validation data
print("Validation Accuracy: {:.4f}".format(val_accuracy))
print("Validation Precision: {:.4f}".format(val_precision))
print("Validation Recall: {:.4f}".format(val_recall))
print("Validation F1 Score: {:.4f}".format(val_f1))


Validation Accuracy: 0.8265
Validation Precision: 0.7770
Validation Recall: 0.7150
Validation F1 Score: 0.7365


In [48]:
params = {'vectorizer__ngram_range': [(1,1),(1,2)],
          'classifier__C': [0.2,0.4],
          'classifier__class_weight': [{"not sexist": 0.7, "sexist": 1}, 'balanced', None]}

pipe_optimized = GridSearchCV(pipe,param_grid=params,cv=10,scoring=make_scorer(f1_score, average="macro"),n_jobs=n_cpu-1,refit=True)
pipe_optimized.fit(x_train, y_train)
print("Migliori iperparametri:",pipe_optimized.best_params_)

Migliori iperparametri: {'classifier__C': 0.2, 'classifier__class_weight': 'balanced', 'vectorizer__ngram_range': (1, 2)}


In [49]:
best_params = pipe_optimized.best_params_
pipe.set_params(**best_params)
pipe.fit(x_train, y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x0000028D25F7B820>),
                ('vectorizer',
                 CountVectorizer(ngram_range=(1, 2),
                                 tokenizer=<bound method TreebankWordTokenizer.tokenize of <nltk.tokenize.treebank.TreebankWordTokenizer object at 0x0000028D267923A0>>)),
                ('classifier',
                 LinearSVC(C=0.2, class_weight='balanced', max_iter=10000))])

In [50]:
y_pred = pipe_optimized.predict(x_test)

# Calcolo delle metriche di valutazione sul set di test

val_accuracy = accuracy_score(y_test, y_pred)
val_precision = precision_score(y_test, y_pred, average="macro")
val_recall = recall_score(y_test, y_pred, average="macro")
val_f1 = f1_score(y_test, y_pred, average="macro")

# Print the evaluation metrics for the validation data

print("Validation Accuracy: {:.4f}".format(val_accuracy))
print("Validation Precision: {:.4f}".format(val_precision))
print("Validation Recall: {:.4f}".format(val_recall))
print("Validation F1 Score: {:.4f}".format(val_f1))

Validation Accuracy: 0.8397
Validation Precision: 0.7944
Validation Recall: 0.7414
Validation F1 Score: 0.7615


# **Task B**

In [61]:
dft = read_csv('../Dataset/Processed datasets/train_category.csv')
dfv = read_csv('../Dataset/Processed datasets/dev_category.csv')
dfs = read_csv('../Dataset/Processed datasets/test_category.csv')

x1_train = dft['text']
y1_train = dft['label_category']
x1_val = dfv['text']
y1_val = dfv['label_category']
x1_test = dfs['text']
y1_test= dfs['label_category']
dft.set_index('ID')
dfv.set_index('ID')
dfs.set_index('ID')
print("TRAIN: \n", y1_train.value_counts(), end="\n\n")
print("VALIDATION: \n", y1_val.value_counts(), end="\n\n")
print("TEST: \n", y1_test.value_counts(), end="\n\n")


TRAIN: 
 2. derogation                               1590
3. animosity                                1165
4. prejudiced discussions                    333
1. threats, plans to harm and incitement     310
Name: label_category, dtype: int64

VALIDATION: 
 2. derogation                               227
3. animosity                                167
4. prejudiced discussions                    48
1. threats, plans to harm and incitement     44
Name: label_category, dtype: int64

TEST: 
 2. derogation                               454
3. animosity                                333
4. prejudiced discussions                    94
1. threats, plans to harm and incitement     89
Name: label_category, dtype: int64



In [27]:
from nltk.stem import WordNetLemmatizer
def treebankWordTokenizer(sentence):

    tokenizer = TreebankWordTokenizer()
    lemmatizer = WordNetLemmatizer()

    # Tokenize the sentence
    tokens = tokenizer.tokenize(sentence)

    # Lemmatize each token
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return lemmatized_tokens
    

In [55]:
from transformers import BertTokenizer
from nltk.tokenize import MWETokenizer
from nltk.tokenize import TreebankWordTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
tokenizer1 = MWETokenizer()
tokenizer2 = TreebankWordTokenizer()

vector1 = CountVectorizer(tokenizer = tokenizer.tokenize, ngram_range=(1,2))
vector2 = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,2))
vector3 = TfidfVectorizer(tokenizer = tokenizer.tokenize, ngram_range=(1,2))
vector4 = TfidfVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,2))

vector5 = CountVectorizer(tokenizer = tokenizer1.tokenize, ngram_range=(1,5))
vector6 = CountVectorizer(tokenizer = treebankWordTokenizer, ngram_range=(1,2))

vector7 = CountVectorizer(tokenizer = kerastokenizer, ngram_range=(1,2))

vector8 = TfidfVectorizer(tokenizer = kerastokenizer, ngram_range=(1,2))

NameError: name 'kerastokenizer' is not defined

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.utils import class_weight
import sklearn.metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, precision_score
from sklearn.neighbors import KNeighborsClassifier


classifier = svm.LinearSVC(max_iter = 10000, class_weight= 'balanced')
# Multi-layer Perceptron classifier
#classifier = neural_network.MLPClassifier(hidden_layer_sizes=(200,),verbose=True,max_iter=1000, shuffle = True, tol = 0.0001,warm_start = True)
# Random Foreset
#classifier = ensemble.RandomForestClassifier(class_weight='balanced',n_estimators=300,criterion='entropy',warm_start=True)

# Create the pipeline

pipe_cat = Pipeline([("cleaner", predictors()),
('vectorizer', vector7),
('classifier', classifier)])

pipe_cat.fit(x1_train, y1_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x0000028D4E9EBE50>),
                ('vectorizer',
                 CountVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function treebankWordTokenizer at 0x0000028D3504BF70>)),
                ('classifier',
                 LinearSVC(class_weight='balanced', max_iter=10000))])

In [30]:
y_pred = pipe_cat.predict(x1_val)

# Calcolo delle metriche di valutazione sul set di test
val_accuracy = accuracy_score(y1_val, y_pred)
val_precision = precision_score(y1_val, y_pred, average="macro")
val_recall = recall_score(y1_val, y_pred, average="macro")
val_f1 = f1_score(y1_val, y_pred, average="macro")

# Print the evaluation metrics for the validation data
print("Validation Accuracy: {:.4f}".format(val_accuracy))
print("Validation Precision: {:.4f}".format(val_precision))
print("Validation Recall: {:.4f}".format(val_recall))
print("Validation F1 Score: {:.4f}".format(val_f1))


Validation Accuracy: 0.5370
Validation Precision: 0.4971
Validation Recall: 0.4182
Validation F1 Score: 0.4348


In [34]:
params = {'vectorizer__ngram_range': [(1,1),(1,2)],
          'classifier__class_weight': ['balanced', None]}

pipe1_optimized = GridSearchCV(pipe_cat,param_grid=params,cv=10,scoring=make_scorer(f1_score, average="macro"),n_jobs=n_cpu-1,refit=True)
pipe1_optimized.fit(x1_train, y1_train)
print("Migliori iperparametri:",pipe1_optimized.best_params_)

Migliori iperparametri: {'classifier__class_weight': 'balanced', 'vectorizer__ngram_range': (1, 2)}


In [35]:
best_params = pipe1_optimized.best_params_
pipe_cat.set_params(**best_params)
pipe_cat.fit(x1_train, y1_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x0000028D4E9EBE50>),
                ('vectorizer',
                 CountVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function treebankWordTokenizer at 0x0000028D3504BF70>)),
                ('classifier',
                 LinearSVC(class_weight='balanced', max_iter=10000))])

In [36]:
y_pred = pipe_cat.predict(x1_test)

# Calcolo delle metriche di valutazione sul set di test
val_accuracy = accuracy_score(y1_test, y_pred)
val_precision = precision_score(y1_test, y_pred, average="macro")
val_recall = recall_score(y1_test, y_pred, average="macro")
val_f1 = f1_score(y1_test, y_pred, average="macro")

# Print the evaluation metrics for the validation data
print("Validation Accuracy: {:.4f}".format(val_accuracy))
print("Validation Precision: {:.4f}".format(val_precision))
print("Validation Recall: {:.4f}".format(val_recall))
print("Validation F1 Score: {:.4f}".format(val_f1))

Validation Accuracy: 0.5052
Validation Precision: 0.4984
Validation Recall: 0.4235
Validation F1 Score: 0.4455
